In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
Train_path = '/content/gdrive/MyDrive/data/dataset/Train'
Val_path = '/content/gdrive/MyDrive/data/dataset/Val'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [4]:
from keras import metrics
from keras.backend import dropout
#CNN based model in keras
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])



In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [6]:
#train from scratch

train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = image.ImageDataGenerator(
    rescale=1./255,
)

test_datagen = image.ImageDataGenerator(
    rescale=1./255,
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 232 images belonging to 2 classes.


In [8]:
train_generator.class_indices


{'covid': 0, 'normal': 1}

In [9]:
val_generator = val_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Val',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [10]:
val_generator.class_indices

{'covid': 0, 'normal': 1}

In [11]:
test_generator = test_datagen.flow_from_directory(
    '/content/gdrive/MyDrive/data/dataset/Test',
    target_size=(224,224),
    class_mode='binary',
    shuffle=False
)

Found 80 images belonging to 2 classes.


In [12]:
test_generator.class_indices

{'covid': 0, 'normal': 1}

In [17]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 2
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
8/8 [==============================] - 57s 7s/step - loss: 0.2853 - accuracy: 0.9052 - val_loss: 0.2847 - val_accuracy: 0.9844
Epoch 2/10
8/8 [==============================] - 55s 8s/step - loss: 0.2999 - accuracy: 0.8750 - val_loss: 0.1991 - val_accuracy: 0.9844
Epoch 3/10
8/8 [==============================] - 55s 7s/step - loss: 0.2410 - accuracy: 0.8966 - val_loss: 0.1506 - val_accuracy: 0.9844
Epoch 4/10
8/8 [==============================] - 54s 7s/step - loss: 0.2329 - accuracy: 0.9224 - val_loss: 0.1164 - val_accuracy: 0.9844
Epoch 5/10
8/8 [==============================] - 58s 7s/step - loss: 0.2072 - accuracy: 0.9310 - val_loss: 0.1420 - val_accuracy: 0.9688
Epoch 6/10
8/8 [==============================] - 56s 7s/step - loss: 0.2286 - accuracy: 0.9095 - val_loss: 0.0822 - val_accuracy: 0.9844
Epoch 7/10
8/8 [==============================] - 54s 7s/step - loss: 0.1745 - accuracy: 0.9440 - val_loss: 0.0811 - val_accuracy: 0.9844
Epoch 8/10
8/8 [==================

In [23]:
prediction = model.predict_generator(test_generator)
#print(prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [24]:
from sklearn.metrics import confusion_matrix
y_true = np.array([0] * 40 + [1] * 40)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)


array([[39,  1],
       [ 0, 40]])